In [17]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [18]:
# Import API key
from API_Keys import g_key

In [19]:
city_df = pd.read_csv("../WeatherPy/city.csv")
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ossora,59.2353,163.0719,273.80,99,100,2.81,RU,1618539171
1,1,Geraldton,-28.7667,114.6000,297.15,29,100,8.23,AU,1618539171
2,2,Doha,25.2867,51.5333,296.77,36,0,5.66,QA,1618539172
3,3,George Town,5.4112,100.3354,299.14,89,20,1.03,MY,1618539087
4,4,Amahai,-3.3333,128.9167,301.65,67,100,1.82,ID,1618538887


In [20]:
gmaps.configure(g_key)